## Part 1: Importing Libraries

In [1]:
from src.modules import data_preprocessing as dp
from src.modules import data_visualization as dv
from src.modules import modeling as md

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

## Part 2: Download Dataset

In [2]:
# Declare the ticker symbol. Here we use 5 stocks "AAPL MSFT AMD GOOG META" as an example.
tickers = ['aapl', 'amd', 'msft', 'meta', 'goog']

In [2]:
# tickers = ['meta']

In [3]:
for ticker in tickers:
    # Download the data from Yahoo Finance
    df = dp.download_data(ticker)

    # clean the data
    df = dp.clean_data(df)

    # split the data into training set and testing set
    train_df, test_df = dp.split_data(df)

    # scale the data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_df)
    test_scaled = scaler.transform(test_df)

    # create the training set and testing set
    X_train, y_train = dp.create_dataset(train_scaled, 60)
    X_test, y_test = dp.create_dataset(test_scaled, 60)

    # reshape the data
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # build the model
    model = md.create_model(X_train=X_train, y_train=y_train)

    # train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

    # save the model
    model.save(f'models/{ticker}_model.h5')


[*********************100%%**********************]  1 of 1 completed
Epoch 1/50
55/55 [==============================] - 24s 204ms/step - loss: 0.0037 - root_mean_squared_error: 0.0605 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0472
Epoch 2/50
55/55 [==============================] - 9s 161ms/step - loss: 4.9401e-04 - root_mean_squared_error: 0.0222 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0387
Epoch 3/50
55/55 [==============================] - 9s 161ms/step - loss: 4.7519e-04 - root_mean_squared_error: 0.0218 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0550
Epoch 4/50
55/55 [==============================] - 8s 150ms/step - loss: 3.9869e-04 - root_mean_squared_error: 0.0200 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0417
Epoch 5/50
55/55 [==============================] - 9s 159ms/step - loss: 4.2416e-04 - root_mean_squared_error: 0.0206 - val_loss: 0.0094 - val_root_mean_squared_error: 0.0970
Epoch 6/50
55/55 [==============================] - 8s

c:\Users\thong.lam\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[*********************100%%**********************]  1 of 1 completed
Epoch 1/50
55/55 [==============================] - 40s 352ms/step - loss: 0.0027 - root_mean_squared_error: 0.0524 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0605
Epoch 2/50
55/55 [==============================] - 14s 252ms/step - loss: 6.2855e-04 - root_mean_squared_error: 0.0251 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0653
Epoch 3/50
55/55 [==============================] - 15s 265ms/step - loss: 6.0580e-04 - root_mean_squared_error: 0.0246 - val_loss: 0.0024 - val_root_mean_squared_error: 0.0488
Epoch 4/50
55/55 [==============================] - 15s 271ms/step - loss: 4.7636e-04 - root_mean_squared_error: 0.0218 - val_loss: 0.0033 - val_root_mean_squared_error: 0.0577
Epoch 5/50
55/55 [==============================] - 15s 266ms/step - loss: 4.7668e-04 - root_mean_squared_error: 0.0218 - val_loss: 0.0024 - val_root_mean_squared_error: 0.0492
Epoch 6/50
55/55 [==============================] 

## Part 4: Data Transformation

In [4]:
print(X_test.shape)

(504, 60, 1)


In [5]:
print(X_train.shape)

(1954, 60, 1)


In [6]:
print(df.shape)

(2518, 1)


In [7]:
# Create the testing data set.
# The testing data set contains the remaining 20% of the data but we have to
# # include the last 60 days from the training data set to predict the first stock price of the test data set
# scaled_data_test = scaled_dataset[num_training_points - window_size:, :]

# # # Create the data sets x_test and y_test
# X_test = []
# y_test = dataset[num_training_points:, :]

# for i in range(60, len(scaled_data_test)):
#     X_test.append(scaled_data_test[i-window_size:i, 0])

# # Convert the values into arrays for easier computation 
# X_test = np.array(X_test)
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
# scaled_data_test = scaled_data[num_training_points - window_size:, :]
#X_test, y_test = convert_to_supervised(scaled_data_test, window_size)

# predict the testing data
predictions = model.predict(X_test)
# predictions = scaler.inverse_transform(predictions)
# y_test = scaler.inverse_transform(y_test)
 
# evaluate result by mse and rmse metrics
mse = np.mean(((predictions - y_test) ** 2))
print("MSE", mse)
print("RMSE", np.sqrt(mse))

16/16 [==============================] - 3s 53ms/step
MSE 0.10199876771360585
RMSE 0.3193724592284154
